# Vidio Sentiment Analyzer : Unveiling User Sentiments in Google Play Reviews

`Vidio Sentiment Analyzer: Mengungkap Sentimen Pengguna dalam Ulasan Google Play` merupakan pendekatan inovatif untuk mengidentifikasi pola-pola utama dalam kumpulan ulasan pengguna. Di era informasi digital yang berkembang pesat seperti sekarang, sangat penting bagi Vidio sebagai platform terkemuka untuk mengetahui dan memahami isu-isu utama yang diungkapkan oleh pengguna.

`Sentiment Analyzer` menggunakan teknik canggih untuk memahami sentimen yang mendominasi dalam ulasan Google Play. Dengan volume feedback pengguna yang besar, menjadi sangat penting bagi Vidio untuk mendapatkan wawasan tentang sentimen pengguna.

Penerapan analisis sentimen memungkinkan Vidio mengkategorikan ulasan pengguna menjadi sentimen positif dan negatif memungkinkan pemahaman yang lebih mendalam tentang pengalaman dan preferensi pengguna. Metrik evaluasi yang relevan dapat mencakup distribusi polaritas sentimen, intensitas sentimen, dan tren sentimen dari waktu ke waktu.

Dalam konteks ini, metrik evaluasi kunci dapat melibatkan metrik-metrik standar analisis sentimen seperti akurasi, precision, recall, dan F1 Score. Metrik-metrik ini memberikan pandangan komprehensif tentang seberapa baik Sentiment Analyzer dalam mengklasifikasikan sentimen pengguna dengan akurat.

Wawasan yang diperoleh dari Sentiment Analyzer dapat memberdayakan Vidio untuk meningkatkan kepuasan pengguna, memprioritaskan perbaikan fitur, dan menanggapi masalah dengan cepat. Dengan fokus pada sentimen pengguna, Vidio dapat secara strategis menyelaraskan upayanya untuk memenuhi harapan pengguna dan menjaga pengalaman pengguna yang positif.

Melalui Vidio Sentiment Analyzer, Vidio dapat menavigasi lanskap dinamis opini pengguna di platform Google Play, memungkinkan pengambilan keputusan yang berbasis informasi dan langkah-langkah proaktif untuk meningkatkan kepuasan pengguna.

# Data Preparation

## Import Library

In [6]:
import pandas as pd
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import pycrfsuite 
# Download data NLTK
nltk.download('stopwords')
nltk.download('wordnet')
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tag import CRFTagger
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint

# import pickle and json file for columns and model file
import pickle
import json
import joblib
import yaml
import src.util as util

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
config_data = util.load_config()

## Data Gathering

Data yang digunakan merupakan data yang diambil atau discrapping dari google play pada aplikasi Vidio mulai tanggal 2020-08-03 - 2023-12-20 sebanyak 10000 rows

In [8]:
#Import data yang sudah dibuat sebelumnya dari proses data preparation
data_preproc = util.pickle_load(config_data["data_preparation_result_path"][1])

In [9]:
# sanity check
data_preproc

,content,sentiment
5575,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif
8625,lumayan bagus tapi tolong di perbaiki gangguan...,Positif
9029,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif
6474,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif
6954,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif
...,...,...
56,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif
626,Saya menyesal berlanggan vidio karena pembayar...,Negatif
1851,tolong apknya di update dong masa gak ada film...,Negatif
59,masa nonton premier league vidionya lebih lamb...,Negatif


## Data Preprocessing

## Cleansing

In [6]:
import re

def clean_text(df, text_field, new_text_field_name):
    # Mengubah teks menjadi huruf kecil
    df[new_text_field_name] = df[text_field].str.lower()  
    # Menhapus tanda baca, emotikon, dan karakter khusus yang tidak dibutuhkan
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))
    # Menhapus angka pada teks
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r"\d+", "", x))
    
    return df

In [7]:
# Memanggil fungsi clean_text
data_preproc = clean_text(data_preproc, 'content', 'cleaned_content')

# Menampilkan hasil
data_preproc

,content,sentiment,cleaned_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...
...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...


## Stopwords Removal

In [31]:
"""
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Mengunduh stopwords bahasa Indonesia dari NLTK
stopwords_indonesia = set(stopwords.words('indonesian'))

# Mengunduh stopwords bahasa Indonesia dari Sastrawi
stopword_factory = StopWordRemoverFactory()
stopword_sastrawi = stopword_factory.create_stop_word_remover()

# Menambahkan stopwords khusus tambahan jika diperlukan
custom_stopwords = set(["tambahan", "lainnya"])

# Menggabungkan semua stopwords
stopwords_total = stopwords_indonesia.union(custom_stopwords)

# Fungsi untuk melakukan stopwords removal
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stopwords_total]
    return ' '.join(filtered_tokens)
    
# Mengaplikasikan stopwords removal pada data
data_preproc['cleaned_content_stopwords'] = data_preproc['cleaned_content'].apply(remove_stopwords)

# Menampilkan hasil
data_preproc['cleaned_content_stopwords']
"""

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords untuk bahasa Indonesia (lakukan sekali saja)
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Daftar stopwords dalam bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

# Fungsi untuk menghapus stopwords dari teks
def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Terapkan fungsi pada kolom yang bersih
data_preproc['cleaned_content_stopwords'] = data_preproc['cleaned_content'].apply(remove_stopwords)

# Tampilkan hasil
data_preproc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,content,sentiment,cleaned_content,cleaned_content_stopwords
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...
...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...


## Tokenisasi Kata

In [9]:
from nltk.tokenize import word_tokenize

# Fungsi untuk tokenisasi teks
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Terapkan fungsi pada kolom yang sudah dihapus stopwords
data_preproc['tokenize_content'] = data_preproc['cleaned_content_stopwords'].apply(tokenize_text)

# Tampilkan hasil
data_preproc

,content,sentiment,cleaned_content,cleaned_content_stopwords,tokenize_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...,"[sumpah, aplikasi, bikin, gw, kesel, udah, gw,..."
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...,"[lumayan, bagus, tolong, perbaiki, gangguan, n..."
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...,"[bagus, banget, lihat, tv, dimn, bepergian, ja..."
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...,"[tampilkan, liga, nntin, laliga, liga, champio..."
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...,"[kayaknya, aplikasi, oona, bagus, yg, kecewa, ..."
...,...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...,"[aplikasi, film, anime, tersedia, film, anime,..."
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...,"[menyesal, berlanggan, vidio, pembayaran, otom..."
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner,"[tolong, apknya, update, gak, film, midnight, ..."
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...,"[nonton, premier, league, vidionya, lambat, si..."


## Postag Text

In [10]:
# Mengunduh model CRF tagger untuk Bahasa Indonesia
ct = CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')

# Fungsi untuk melakukan tokenisasi dan postagging
def postag_text(text):
    # Tokenisasi teks
    tokens = word_tokenize(text)

    # Melakukan postagging menggunakan CRFTagger
    postags = ct.tag_sents([tokens])

    return postags[0]  # Kembalikan hasil postag untuk teks pertama

# Terapkan fungsi pada kolom yang telah dihapus stopwords
data_preproc['pos_tags_content'] = data_preproc['cleaned_content_stopwords'].apply(postag_text)

# Tampilkan hasil
data_preproc

,content,sentiment,cleaned_content,cleaned_content_stopwords,tokenize_content,pos_tags_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...,"[sumpah, aplikasi, bikin, gw, kesel, udah, gw,...","[(sumpah, NN), (aplikasi, NN), (bikin, FW), (g..."
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...,"[lumayan, bagus, tolong, perbaiki, gangguan, n...","[(lumayan, NN), (bagus, JJ), (tolong, VB), (pe..."
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...,"[bagus, banget, lihat, tv, dimn, bepergian, ja...","[(bagus, JJ), (banget, NN), (lihat, VB), (tv, ..."
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...,"[tampilkan, liga, nntin, laliga, liga, champio...","[(tampilkan, VB), (liga, CD), (nntin, NN), (la..."
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...,"[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...","[(kayaknya, RB), (aplikasi, VB), (oona, NN), (..."
...,...,...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...,"[aplikasi, film, anime, tersedia, film, anime,...","[(aplikasi, NN), (film, NN), (anime, NN), (ter..."
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...,"[menyesal, berlanggan, vidio, pembayaran, otom...","[(menyesal, NN), (berlanggan, NN), (vidio, NN)..."
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner,"[tolong, apknya, update, gak, film, midnight, ...","[(tolong, VB), (apknya, RB), (update, FW), (ga..."
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...,"[nonton, premier, league, vidionya, lambat, si...","[(nonton, FW), (premier, FW), (league, FW), (v..."


## get selected tags (Indexed Content)

In [11]:
def get_selected_tags(tokens_with_tags):
    selected_tags = ['NN', 'VB', 'JJ', 'RB']
    return [text for text, tag in tokens_with_tags if tag in selected_tags]

# Terapkan fungsi pada kolom 'postag_content'
data_preproc['indexed_content'] = data_preproc['pos_tags_content'].apply(get_selected_tags)

data_preproc

,content,sentiment,cleaned_content,cleaned_content_stopwords,tokenize_content,pos_tags_content,indexed_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...,"[sumpah, aplikasi, bikin, gw, kesel, udah, gw,...","[(sumpah, NN), (aplikasi, NN), (bikin, FW), (g...","[sumpah, aplikasi, ulang, aja, ga, dibuka, buk..."
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...,"[lumayan, bagus, tolong, perbaiki, gangguan, n...","[(lumayan, NN), (bagus, JJ), (tolong, VB), (pe...","[lumayan, bagus, tolong, perbaiki, gangguan, n..."
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...,"[bagus, banget, lihat, tv, dimn, bepergian, ja...","[(bagus, JJ), (banget, NN), (lihat, VB), (tv, ...","[bagus, banget, lihat, tv, dimn, bepergian, ja..."
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...,"[tampilkan, liga, nntin, laliga, liga, champio...","[(tampilkan, VB), (liga, CD), (nntin, NN), (la...","[tampilkan, nntin, tpi, mala, tdk, alasannya, ..."
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...,"[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...","[(kayaknya, RB), (aplikasi, VB), (oona, NN), (...","[kayaknya, aplikasi, oona, bagus, yg, kecewa, ..."
...,...,...,...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...,"[aplikasi, film, anime, tersedia, film, anime,...","[(aplikasi, NN), (film, NN), (anime, NN), (ter...","[aplikasi, film, anime, tersedia, film, anime,..."
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...,"[menyesal, berlanggan, vidio, pembayaran, otom...","[(menyesal, NN), (berlanggan, NN), (vidio, NN)...","[menyesal, berlanggan, vidio, pembayaran, otom..."
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner,"[tolong, apknya, update, gak, film, midnight, ...","[(tolong, VB), (apknya, RB), (update, FW), (ga...","[tolong, apknya]"
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...,"[nonton, premier, league, vidionya, lambat, si...","[(nonton, FW), (premier, FW), (league, FW), (v...","[vidionya, lambat, situs, ilegal, official, pa..."


## Lemma Content

In [12]:
# Menggabungkan lema ke dalam teks
data_preproc['lemmas_to_content'] = data_preproc['indexed_content'].apply(lambda lemmas: ' '.join(map(str, lemmas)))

data_preproc

,content,sentiment,cleaned_content,cleaned_content_stopwords,tokenize_content,pos_tags_content,indexed_content,lemmas_to_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...,"[sumpah, aplikasi, bikin, gw, kesel, udah, gw,...","[(sumpah, NN), (aplikasi, NN), (bikin, FW), (g...","[sumpah, aplikasi, ulang, aja, ga, dibuka, buk...",sumpah aplikasi ulang aja ga dibuka buka muncu...
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...,"[lumayan, bagus, tolong, perbaiki, gangguan, n...","[(lumayan, NN), (bagus, JJ), (tolong, VB), (pe...","[lumayan, bagus, tolong, perbaiki, gangguan, n...",lumayan bagus tolong perbaiki gangguan nya cha...
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...,"[bagus, banget, lihat, tv, dimn, bepergian, ja...","[(bagus, JJ), (banget, NN), (lihat, VB), (tv, ...","[bagus, banget, lihat, tv, dimn, bepergian, ja...",bagus banget lihat tv dimn bepergian jarak jau...
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...,"[tampilkan, liga, nntin, laliga, liga, champio...","[(tampilkan, VB), (liga, CD), (nntin, NN), (la...","[tampilkan, nntin, tpi, mala, tdk, alasannya, ...",tampilkan nntin tpi mala tdk alasannya hak cip...
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...,"[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...","[(kayaknya, RB), (aplikasi, VB), (oona, NN), (...","[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...",kayaknya aplikasi oona bagus yg kecewa dgn apl...
...,...,...,...,...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...,"[aplikasi, film, anime, tersedia, film, anime,...","[(aplikasi, NN), (film, NN), (anime, NN), (ter...","[aplikasi, film, anime, tersedia, film, anime,...",aplikasi film anime tersedia film anime tersed...
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...,"[menyesal, berlanggan, vidio, pembayaran, otom...","[(menyesal, NN), (berlanggan, NN), (vidio, NN)...","[menyesal, berlanggan, vidio, pembayaran, otom...",menyesal berlanggan vidio pembayaran otomatis ...
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner,"[tolong, apknya, update, gak, film, midnight, ...","[(tolong, VB), (apknya, RB), (update, FW), (ga...","[tolong, apknya]",tolong apknya
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...,"[nonton, premier, league, vidionya, lambat, si...","[(nonton, FW), (premier, FW), (league, FW), (v...","[vidionya, lambat, situs, ilegal, official, pa...",vidionya lambat situs ilegal official partner ...


## Stemming Data kolom yang sudah di Tokenisasi

In [14]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [15]:
#-----------------STEMMING -----------------
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# proses stemming
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
hitung=0

for document in data_preproc['tokenize_content']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    hitung+=1
    print(hitung,":",term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

17375
------------------------
1 : sumpah : sumpah
2 : aplikasi : aplikasi
3 : bikin : bikin
4 : gw : gw
5 : kesel : kesel
6 : udah : udah
7 : hapus : hapus
8 : trs : trs
9 : download : download
10 : ulang : ulang
11 : aja : aja
12 : ga : ga
13 : dibuka : buka
14 : buka : buka
15 : munculnya : muncul
16 : layar : layar
17 : putih : putih
18 : doang : doang
19 : gatau : gatau
20 : gada : gada
21 : kendala : kendala
22 : lumayan : lumayan
23 : bagus : bagus
24 : tolong : tolong
25 : perbaiki : baik
26 : gangguan : ganggu
27 : nya : nya
28 : channel : channel
29 : putar : putar
30 : banget : banget
31 : lihat : lihat
32 : tv : tv
33 : dimn : dimn
34 : bepergian : pergi
35 : jarak : jarak
36 : jauhdekat : jauhdekat
37 : mati : mati
38 : lampulistrik : lampulistrik
39 : dlln : dlln
40 : tampilkan : tampil
41 : liga : liga
42 : nntin : nntin
43 : laliga : laliga
44 : champion : champion
45 : tpi : tpi
46 : mala : mala
47 : tdk : tdk
48 : alasannya : alas
49 : hak : hak
50 : cipta : cipta
51 

In [16]:
#memisahkan file eksekusinya setelah pembacaaan term selesai
data_preproc['stemmed_content'] = data_preproc['tokenize_content'].apply(lambda x:' '.join(get_stemmed_term(x)))
data_preproc

,content,sentiment,cleaned_content,cleaned_content_stopwords,tokenize_content,pos_tags_content,indexed_content,lemmas_to_content,stemmed_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,Negatif,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...,"[sumpah, aplikasi, bikin, gw, kesel, udah, gw,...","[(sumpah, NN), (aplikasi, NN), (bikin, FW), (g...","[sumpah, aplikasi, ulang, aja, ga, dibuka, buk...",sumpah aplikasi ulang aja ga dibuka buka muncu...,sumpah aplikasi bikin gw kesel udah gw hapus t...
1,lumayan bagus tapi tolong di perbaiki gangguan...,Positif,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...,"[lumayan, bagus, tolong, perbaiki, gangguan, n...","[(lumayan, NN), (bagus, JJ), (tolong, VB), (pe...","[lumayan, bagus, tolong, perbaiki, gangguan, n...",lumayan bagus tolong perbaiki gangguan nya cha...,lumayan bagus tolong baik ganggu nya channel p...
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,Positif,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...,"[bagus, banget, lihat, tv, dimn, bepergian, ja...","[(bagus, JJ), (banget, NN), (lihat, VB), (tv, ...","[bagus, banget, lihat, tv, dimn, bepergian, ja...",bagus banget lihat tv dimn bepergian jarak jau...,bagus banget lihat tv dimn pergi jarak jauhdek...
3,"Masa tidak bisa tampilkan semua liga, kami mau...",Negatif,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...,"[tampilkan, liga, nntin, laliga, liga, champio...","[(tampilkan, VB), (liga, CD), (nntin, NN), (la...","[tampilkan, nntin, tpi, mala, tdk, alasannya, ...",tampilkan nntin tpi mala tdk alasannya hak cip...,tampil liga nntin laliga liga champion tpi mal...
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",Negatif,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...,"[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...","[(kayaknya, RB), (aplikasi, VB), (oona, NN), (...","[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...",kayaknya aplikasi oona bagus yg kecewa dgn apl...,kayak aplikasi oona bagus yg kecewa dgn aplika...
...,...,...,...,...,...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,Negatif,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...,"[aplikasi, film, anime, tersedia, film, anime,...","[(aplikasi, NN), (film, NN), (anime, NN), (ter...","[aplikasi, film, anime, tersedia, film, anime,...",aplikasi film anime tersedia film anime tersed...,aplikasi film anime sedia film anime sedia apl...
9996,Saya menyesal berlanggan vidio karena pembayar...,Negatif,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...,"[menyesal, berlanggan, vidio, pembayaran, otom...","[(menyesal, NN), (berlanggan, NN), (vidio, NN)...","[menyesal, berlanggan, vidio, pembayaran, otom...",menyesal berlanggan vidio pembayaran otomatis ...,sesal langgan vidio bayar otomatis potong dana...
9997,tolong apknya di update dong masa gak ada film...,Negatif,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner,"[tolong, apknya, update, gak, film, midnight, ...","[(tolong, VB), (apknya, RB), (update, FW), (ga...","[tolong, apknya]",tolong apknya,tolong apknya update gak film midnight runner
9998,masa nonton premier league vidionya lebih lamb...,Negatif,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...,"[nonton, premier, league, vidionya, lambat, si...","[(nonton, FW), (premier, FW), (league, FW), (v...","[vidionya, lambat, situs, ilegal, official, pa...",vidionya lambat situs ilegal official partner ...,nonton premier league vidionya lambat situs il...


In [17]:
# Ganti nilai 'Negatif' dengan 1 dan 'Positif' dengan 0 pada kolom 'sentiment'
data_preproc['sentiment'] = data_preproc['sentiment'].replace({'Negatif': 1, 'Positif': 0})

# Tampilkan hasil
data_preproc

,content,sentiment,cleaned_content,cleaned_content_stopwords,tokenize_content,pos_tags_content,indexed_content,lemmas_to_content,stemmed_content
0,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,1,sumpah aplikasi ini bikin gw kesel sendiri uda...,sumpah aplikasi bikin gw kesel udah gw hapus t...,"[sumpah, aplikasi, bikin, gw, kesel, udah, gw,...","[(sumpah, NN), (aplikasi, NN), (bikin, FW), (g...","[sumpah, aplikasi, ulang, aja, ga, dibuka, buk...",sumpah aplikasi ulang aja ga dibuka buka muncu...,sumpah aplikasi bikin gw kesel udah gw hapus t...
1,lumayan bagus tapi tolong di perbaiki gangguan...,0,lumayan bagus tapi tolong di perbaiki gangguan...,lumayan bagus tolong perbaiki gangguan nya cha...,"[lumayan, bagus, tolong, perbaiki, gangguan, n...","[(lumayan, NN), (bagus, JJ), (tolong, VB), (pe...","[lumayan, bagus, tolong, perbaiki, gangguan, n...",lumayan bagus tolong perbaiki gangguan nya cha...,lumayan bagus tolong baik ganggu nya channel p...
2,Bagus banget Jadi bisa lihat TV dimn pun saat ...,0,bagus banget jadi bisa lihat tv dimn pun saat ...,bagus banget lihat tv dimn bepergian jarak jau...,"[bagus, banget, lihat, tv, dimn, bepergian, ja...","[(bagus, JJ), (banget, NN), (lihat, VB), (tv, ...","[bagus, banget, lihat, tv, dimn, bepergian, ja...",bagus banget lihat tv dimn bepergian jarak jau...,bagus banget lihat tv dimn pergi jarak jauhdek...
3,"Masa tidak bisa tampilkan semua liga, kami mau...",1,masa tidak bisa tampilkan semua liga kami mau ...,tampilkan liga nntin laliga liga champion tpi ...,"[tampilkan, liga, nntin, laliga, liga, champio...","[(tampilkan, VB), (liga, CD), (nntin, NN), (la...","[tampilkan, nntin, tpi, mala, tdk, alasannya, ...",tampilkan nntin tpi mala tdk alasannya hak cip...,tampil liga nntin laliga liga champion tpi mal...
4,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",1,kayaknya aplikasi oona lebih bagus bagi yg kec...,kayaknya aplikasi oona bagus yg kecewa dgn apl...,"[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...","[(kayaknya, RB), (aplikasi, VB), (oona, NN), (...","[kayaknya, aplikasi, oona, bagus, yg, kecewa, ...",kayaknya aplikasi oona bagus yg kecewa dgn apl...,kayak aplikasi oona bagus yg kecewa dgn aplika...
...,...,...,...,...,...,...,...,...,...
9995,Aplikasi dari 2015 tapi film dan anime yang te...,1,aplikasi dari tapi film dan anime yang tersed...,aplikasi film anime tersedia film anime tersed...,"[aplikasi, film, anime, tersedia, film, anime,...","[(aplikasi, NN), (film, NN), (anime, NN), (ter...","[aplikasi, film, anime, tersedia, film, anime,...",aplikasi film anime tersedia film anime tersed...,aplikasi film anime sedia film anime sedia apl...
9996,Saya menyesal berlanggan vidio karena pembayar...,1,saya menyesal berlanggan vidio karena pembayar...,menyesal berlanggan vidio pembayaran otomatis ...,"[menyesal, berlanggan, vidio, pembayaran, otom...","[(menyesal, NN), (berlanggan, NN), (vidio, NN)...","[menyesal, berlanggan, vidio, pembayaran, otom...",menyesal berlanggan vidio pembayaran otomatis ...,sesal langgan vidio bayar otomatis potong dana...
9997,tolong apknya di update dong masa gak ada film...,1,tolong apknya di update dong masa gak ada film...,tolong apknya update gak film midnight runner,"[tolong, apknya, update, gak, film, midnight, ...","[(tolong, VB), (apknya, RB), (update, FW), (ga...","[tolong, apknya]",tolong apknya,tolong apknya update gak film midnight runner
9998,masa nonton premier league vidionya lebih lamb...,1,masa nonton premier league vidionya lebih lamb...,nonton premier league vidionya lambat situs il...,"[nonton, premier, league, vidionya, lambat, si...","[(nonton, FW), (premier, FW), (league, FW), (v...","[vidionya, lambat, situs, ilegal, official, pa...",vidionya lambat situs ilegal official partner ...,nonton premier league vidionya lambat situs il...


## Data Splitting

In [19]:
from sklearn.model_selection import train_test_split

X = data_preproc['stemmed_content']
y = data_preproc['sentiment']

In [20]:
X

0       sumpah aplikasi bikin gw kesel udah gw hapus t...
1       lumayan bagus tolong baik ganggu nya channel p...
2       bagus banget lihat tv dimn pergi jarak jauhdek...
3       tampil liga nntin laliga liga champion tpi mal...
4       kayak aplikasi oona bagus yg kecewa dgn aplika...
                              ...                        
9995    aplikasi film anime sedia film anime sedia apl...
9996    sesal langgan vidio bayar otomatis potong dana...
9997        tolong apknya update gak film midnight runner
9998    nonton premier league vidionya lambat situs il...
9999                                 bagus aplikasi iklan
Name: stemmed_content, Length: 10000, dtype: object

In [21]:
y

0       1
1       0
2       0
3       1
4       1
       ..
9995    1
9996    1
9997    1
9998    1
9999    1
Name: sentiment, Length: 10000, dtype: int64

In [22]:
y.value_counts()

sentiment
1    7461
0    2539
Name: count, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
#Split Data 70% training 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 123,
                                                    stratify=y)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [24]:
X_train

2166    bantu orang yg gx tv lihat chanel yg suka yg l...
6196    streaming nya si ok ga lag kecawa bgt pas stre...
5211    acara bola liga ingris pra laga pakai bandit b...
1867    kenal nama bunga kls suka sih salah vidio gk b...
5858    ni aplikasi latih sabar seru seru nya nonton t...
                              ...                        
9456                     cuman fokus serial tv film jadul
1735    mohon maaf ksh bintang iklan mohon rang iklann...
2560    gimana ni udah langgan jam doang habis loading...
3359    gk enak nonton ikat cinta tanggal nya gk trs t...
2718    paksa bintang nawarin promo guna tp tunggu pro...
Name: stemmed_content, Length: 7000, dtype: object

In [25]:
X_test

6709    baguss tolonglahh iklan banyaksetiap seru nya ...
4882    premium bayar eh payah loading live nya banget...
7367    bandingin ama yutub platform app iklan banyakm...
3340    dlu sblm upgradeaplikasi bs floating hp ku dgn...
4983    iklan masuk tayang iklan masuk tayang skip ikl...
                              ...                        
3626    beli paket proses nya tdk trus pake beli paket...
145     aja video simpan video aplikasi putar sy coba ...
595     asli kasih bintang gajadi salur antv admin tol...
8161    pilih tonton filmsport dll aplikasi vidio liha...
8533    udah mantaappmin tambahin film kartun anak yg ...
Name: stemmed_content, Length: 3000, dtype: object

In [26]:
y_train

2166    0
6196    1
5211    1
1867    1
5858    1
       ..
9456    1
1735    1
2560    1
3359    1
2718    1
Name: sentiment, Length: 7000, dtype: int64

In [27]:
y_test

6709    1
4882    1
7367    1
3340    1
4983    1
       ..
3626    0
145     1
595     1
8161    0
8533    0
Name: sentiment, Length: 3000, dtype: int64

In [28]:
# Split data train menjadi train dan validation set
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, 
                                                    test_size=0.4, 
                                                    random_state=42,
                                                    stratify = y_test
                                                   )

C:\Users\hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [29]:
X_test

7025                 sayah udah langgan iklan tolong baik
149     bagus nih channel axn animax nya coming soon t...
7067    knp ya download magic gagal pas download yg ko...
3715    bagus utk nonton film chanel tv iklan lumayan ...
2971    aplikasi bagus tpi haya gk lengkap masak nonto...
                              ...                        
8961    bingung beli paket platinum via indosat n goog...
2798    masuk homepage tulis failed to load comeback i...
9700    aplikasi nya gagal tampil halaman sinyal bagus...
1954    suka aplikasi tolong serial jodha akbar lengka...
6168    aplikasi sial gak andal final buka gak tonton ...
Name: stemmed_content, Length: 1800, dtype: object

In [30]:
X_valid

6366    bagus sih g lengkap film kamen rider black sun...
500     langgan premier tp tulis capai maksimal perang...
8740    apl nya bagus ga kendala gampang beli fitur pr...
1304    eheheh udah bintang tuh mueheeee aplikasi bagu...
2356    aplikasi oke aja thanks ya vidiocuman klo dido...
                              ...                        
7064    min dowload film download kga download dowload...
1763    aps vidio bagus kekuranganyayaitu koleksi drak...
9620                             g nonton bola beli paket
2550    nonton vidiolagi seru eh iklan gitu aja iklan ...
6710    heran ni aplikasi pake data lot nya ampunpadah...
Name: stemmed_content, Length: 1200, dtype: object

In [31]:
y_test.value_counts()

sentiment
1    1343
0     457
Name: count, dtype: int64

In [32]:
y_valid.value_counts()

sentiment
1    895
0    305
Name: count, dtype: int64

## TFIDF

In [33]:
X_train = X_train.reset_index()
X_test  = X_test.reset_index()
X_valid = X_valid.reset_index()

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import pickle

# Inisialisasi dan fit TfidfVectorizer
def fit_tfidf_vectorizer(X_train):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer.fit(X_train['stemmed_content'])
    
    # Simpan TfidfVectorizer
    with open('model/tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    
    return tfidf_vectorizer

In [35]:
# Load TfidfVectorizer dari file
def load_tfidf_vectorizer(file_path):
    with open(file_path, 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    return tfidf_vectorizer

In [36]:
X_train_fit = fit_tfidf_vectorizer(X_train)
X_train_fit

TfidfVectorizer()

In [37]:
# Memuat TfidfVectorizer dari file
tfidf_vectorizer = load_tfidf_vectorizer('model/tfidf_vectorizer.pkl')

In [38]:
# Transformasi data pelatihan
X_train_tfidf = tfidf_vectorizer.transform(X_train['stemmed_content'])
# Transformasi data uji
X_test_tfidf = tfidf_vectorizer.transform(X_test['stemmed_content'])
# Transformasi data validasi
X_valid_tfidf = tfidf_vectorizer.transform(X_valid['stemmed_content'])

In [39]:
X_train_tfidf

<7000x11664 sparse matrix of type '<class 'numpy.float64'>'
	with 108136 stored elements in Compressed Sparse Row format>

In [40]:
X_test_tfidf

<1800x11664 sparse matrix of type '<class 'numpy.float64'>'
	with 25534 stored elements in Compressed Sparse Row format>

In [41]:
X_valid_tfidf

<1200x11664 sparse matrix of type '<class 'numpy.float64'>'
	with 17107 stored elements in Compressed Sparse Row format>

## Save Data

In [42]:
util.pickle_dump(X_train_tfidf, config_data["train_tfidf_set_path"][0])
util.pickle_dump(y_train, config_data["train_tfidf_set_path"][1])
    
util.pickle_dump(X_valid_tfidf, config_data["valid_tfidf_set_path"][0])
util.pickle_dump(y_valid, config_data["valid_tfidf_set_path"][1])

util.pickle_dump(X_test_tfidf, config_data["test_tfidf_set_path"][0])
util.pickle_dump(y_test, config_data["test_tfidf_set_path"][1])